## Make sure we have our dependencies

### Now, because we have a Dockerfile for our development environment... this should just work...
To run your determined Jupyter notebook with this image...

```yaml
## replace this 
environment:
  add_capabilities: null
  drop_capabilities: null
  environment_variables: {}
  force_pull_image: false
  image:
    cpu: determinedai/environments:py-3.8-pytorch-1.10-tf-2.8-cpu-9119094
    cuda: determinedai/environments:cuda-11.3-pytorch-1.10-tf-2.8-gpu-9119094 ### this guy right here

##replace it with this 

    cuda: katestephens/kate-nemo:<version>

## you're welcome :)
```


### If you don't want to use Jupyter in Determined and you just wanna use VSCode in a Determined shell... do the below
create a `determined_config.yaml` in your homedir on your local machine with the following
```yaml
bind_mounts:
  - container_path: /run/determined/workdir/shared_fs
    host_path: /mnt/mapr_nfs/<path_to_det_share>/determined/det_share
    propagation: rprivate
    read_only: false
  - container_path: /determined_shared_fs
    host_path: /mnt/mapr_nfs/<path_to_det_checkpoints>/determined/det_checkpoints
    propagation: rprivate
    read_only: false
  - container_path: /mnt/mapr_nfs
    host_path: /mnt/mapr_nfs
    propagation: rprivate
    read_only: false
debug: false
description: my-awesome-nb
entrypoint: null
environment:
  add_capabilities: null
  drop_capabilities: null
  environment_variables: {}
  force_pull_image: false
  image:
    cpu: determinedai/environments:py-3.8-pytorch-1.10-tf-2.8-cpu-9119094
    cuda: katestephens/kate-nemo:<version>
  pod_spec: null
  ports: null
  slurm: null
idle_timeout: null
notebook_idle_type: kernels_or_terminals
resources:
  agent_label: ''
  devices: null
  resource_pool: kubernetes
  slots: 1
  weight: 1
work_dir: null
```
Now... you'll want to make sure you have the mapping to det master in your local dev environment
```bash
echo $DET_MASTER
```

if nothing shows up...
`export DET_MASTER=<ipaddrOfDet:port>`
better yet... add it to your `~/.bashrc` and `source ~/.bashrc`

To test that you're really talking to `DET_MASTER` run `det shell list` from your terminal

To start a shell with the config/docker image loaded from YOUR terminal run
```bash
det shell start --show-ssh-command --config-file ~/determined_config.yaml
```

Then add the ssh host to your vscode instance (https://docs.determined.ai/latest/interfaces/ide-integration.html#visual-studio-code)

### Last Step... we wanna check our dependencies... especially apex...

In [6]:
#make sure we have all our dependencies
import os
from os.path import exists, join, basename, splitext

if not exists('apex'):
 !pip list | grep apex
 !echo "did I return apex?! If not... oopsie no Apex found uncomment the lines in this block"

!git clone https://github.com/NVIDIA/apex
!cd ./apex && pip install -v --disable-pip-version-check --no-cache-dir --global-option="--cpp_ext" --global-option="--cuda_ext" ./

did I return apex?! If not... oopsie no Apex found uncomment the lines in this block
Cloning into 'apex'...
remote: Enumerating objects: 10703, done.
remote: Counting objects: 100% (278/278), done.
remote: Compressing objects: 100% (178/178), done.
remote: Total 10703 (delta 169), reused 168 (delta 100), pack-reused 10425
Receiving objects: 100% (10703/10703), 15.21 MiB | 980.00 KiB/s, done.
Resolving deltas: 100% (7360/7360), done.
Using pip 22.3.1 from /Users/kthorpe/miniconda3/envs/nemo/lib/python3.9/site-packages/pip (python 3.9)
DEPRECATION: --no-binary currently disables reading from the cache of locally built wheels. In the future --no-binary will not influence the wheel cache. pip 23.1 will enforce this behaviour change. A possible replacement is to use the --no-cache-dir option. You can use the flag --use-feature=no-binary-enable-wheel-cache to test the upcoming behaviour. Discussion can be found at https://github.com/pypa/pip/issues/11453
Processing /Users/kthorpe/src/UIforML